In [1]:
from IPython.display import HTML, display
def set_css():
  display(HTML('''
  <style>
    pre {white-space: pre-wrap;}
  </style>
  '''))
get_ipython().events.register("pre_run_cell", set_css)
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
import os
import sys
try:
    my_path = "/content/notebooks"
    os.symlink("/content/drive/MyDrive/ColabNotebooks/my_env", my_path)
    sys.path.insert(0, my_path)
except:
    pass
os.chdir(my_path)
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, Bidirectional,\
TimeDistributed, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import Model
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
from transformers import BertModel
from transformers import TFBertModel
from tokenization_kobert import KoBertTokenizer
import json
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import os
import urllib.request
import logging
import sklearn

logging.basicConfig(level=logging.ERROR)

Mounted at /content/drive


In [2]:
# urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt",
#                            filename="ratings_train.txt")
# urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt",
#                            filename="ratings_test.txt")

ratings_train = pd.read_table("ratings_train.txt")
ratings_test = pd.read_table("ratings_test.txt")


- 문장을 토크나이징 하면 "전율을 일으키는 영화. 다시 보고싶은 영화"가 `['▁전', '율을', '▁일으키', '는', '▁영화', '.', '▁다시', '▁보고', '싶', '은', '▁영화']`로 토크나이징이 됩니다.  
- 이것을 버트 인풋에 들어갈 숫자로 바꿔주면 `[2, 4012, 7071, 3815, 5760, 3394, 54, 1574, 2358, 6751, 7086, 3394, 3]`로 바뀌게 됩니다. 여기 나오는 숫자들이 버트 인풋에 들어가는 token input입니다.
- 버트 모형에 들어가는 인풋은 일정한 길이를 가져야 합니다.(본 예제에서는 64)
따라서 남는 부분은 1로 채워지게 됩니다(패딩)
- Segment Embeddings는 문장이 앞문장인지 뒷문장인지 구분해주는 역할을 합니다.본 문장에서는 문장 하나만 Input으로 들어가기 때문에 0만 들어가게 되고, 문장 길이만큼의 0이 Input으로 들어가게 됩니다.
- 마스크 인풋은 토큰 인풋에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 두게 됩니다.  
- 종합하면,
버트의 인풋은 토큰, 세그먼트, 마스크로 이루어져 있습니다. huggingface에서는 순서가 [토큰 인풋, 마스크 인풋, 세그먼트 인풋] 입니다.

In [3]:
seq_len = 64

model = BertModel.from_pretrained("monologg/kobert")
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")

def load_data(df):
    def convert_data(df):
        tokens, masks, segments, labels = [], [], [], []
        for _, row in tqdm(df.iterrows()):
            token = tokenizer.encode(row[data_column], max_length=seq_len,
                                    pad_to_max_length=True, truncation=True)
            n_zeros = token.count(1)
            mask = [1]*(seq_len - n_zeros) + [0]*n_zeros
            # 문장의 전후관계를 구분해주는 세그먼트는 문장이 1개밖에 없으므로 모두 0
            segment = [0]*seq_len

            tokens.append(token)
            masks.append(mask)
            segments.append(segment)
            labels.append(row[label_column])
        tokens = np.array(tokens)
        masks = np.array(masks)
        segments = np.array(segments)
        labels = np.array(labels)
        return [tokens, masks, segments], labels

    df[data_column] = df[data_column].astype("str")
    df[label_column] = df[label_column].astype("int")
    X, y = convert_data(df)
    return X, y

data_column = "document"
label_column = "label"

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

In [4]:
train_X, train_y = load_data(ratings_train)
test_X, test_y = load_data(ratings_test)

0it [00:00, ?it/s]/content/notebooks/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
150000it [00:56, 2638.30it/s]
50000it [00:17, 2848.21it/s]


In [10]:
def create_model():
    input_token = Input((seq_len,), dtype=tf.int32, name="input_token")
    input_mask = Input((seq_len,), dtype=tf.int32, name="input_mask")
    input_segment = Input((seq_len,), dtype=tf.int32, name="input_segment")
    inputs = [input_token, input_mask, input_segment]

    model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True)
    bert_outputs = model(inputs)
    dropout = Dropout(rate=0.5, name="dropout")(bert_outputs[1])
    dense = Dense(units=1, activation="sigmoid", 
                kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02),
                name="dense")(dropout)
    sentiment_model = Model(inputs=inputs, outputs=dense)

    sentiment_model.summary()

    # `total_steps` : batch_size*n_epochs
    opt = tfa.optimizers.RectifiedAdam(lr=5.0e-5, total_steps=2344*4, warmup_proportion=0.1,
                                    min_lr=1e-5, epsilon=1e-08, clipnorm=1.0)
    sentiment_model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["binary_accuracy"])
    return sentiment_model

In [30]:
model_path = "sktkobert_navermoviereviewdataset.h5"
if os.path.exists(model_path):
    sentiment_model = create_model()
    sentiment_model.load_weights(model_path)
else:
    # resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="grpc://" + os.environ["COLAB_TPU_ADDR"])
    # tf.config.experimental_connect_to_cluster(resolver)
    # tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.experimental.TPUStrategy(resolver)
    with strategy.scope():
        sentiment_model = create_model()

        hist = sentiment_model.fit(x=train_X, y=train_y, validation_split=0.2, batch_size=64, epochs=3, verbose=1, shuffle=True)

        sentiment_model.save_weights(model_path, overwrite=True)

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 64)]         0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 64)]         0                                            
__________________________________________________________________________________________________
input_segment (InputLayer)      [(None, 64)]         0                                            
__________________________________________________________________________________________________
tf_bert_model_3 (TFBertModel)   ((None, 64, 768), (N 92186880    input_token[0][0]                
                                                                 input_mask[0][0]           

In [17]:
def predict_convert_data(data_df):
    tokens, masks, segments = [], [], []
    for _, row in data_df.iterrows():
        # token : 문장을 토큰화함
        token = tokenizer.encode(row[data_column], max_length=seq_len,
                                 padding="max_length")
        n_zeros = token.count(1)
        mask = [1]*(seq_len - n_zeros) + [0]*n_zeros
        # 문장의 전후관계를 구분해주는 세그먼트는 문장이 1개밖에 없으므로 모두 0
        segment = [0]*seq_len

        tokens.append(token)
        masks.append(mask)
        segments.append(segment)  
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)

    return [tokens, masks, segments]

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def predict_load_data(df):
    df[data_column] = df[data_column].astype("str")
    X = predict_convert_data(df)
    return X

In [18]:
preds = sentiment_model.predict(test_X)

In [19]:
test_set = predict_load_data(ratings_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


In [21]:
y_true = test["label"]
# F1 Score 확인
print(classification_report(y_true, np.round(preds,0)))

NameError: ignored

In [22]:
def sentence_convert_data(data):
    global tokenizer
    tokens, masks, segments = [], [], []
    token = tokenizer.encode(data, max_length=seq_len, pad_to_max_length=True)
    
    n_zeros = token.count(0) 
    mask = [1]*(seq_len-n_zeros) + [0]*n_zeros 
    segment = [0]*seq_len

    tokens.append(token)
    segments.append(segment)
    masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

def movie_evaluation_predict(sentence):
    data_x = sentence_convert_data(sentence)
    predict = sentiment_model.predict(data_x)
    predict_value = np.ravel(predict)
    predict_answer = np.round(predict_value,0).item()
    
    if predict_answer == 0:
      print("(부정 확률 : %.2f) 부정적인 영화 평가입니다." % (1-predict_value))
    elif predict_answer == 1:
      print("(긍정 확률 : %.2f) 긍정적인 영화 평가입니다." % predict_value)

In [28]:
movie_evaluation_predict("즐겁습니다")

/content/notebooks/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


(부정 확률 : 0.64) 부정적인 영화 평가입니다.
